In [2]:
import pandas as pd
import numpy as np
import time
import re
import pickle
import time

In [3]:
tracking2018 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2018.csv")
tracking2019 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2019.csv")
tracking2020 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2020.csv")

tracking2018.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right


In [4]:
tracking2018.query("displayName == 'football' and gameId == 2018123000 and playId == 36")

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
2310,2018-12-30T21:25:32.200,45.45,25.96,0.00,0.00,0.00,NaN,NaN,None,NaN,football,NaN,NaN,football,1,2018123000,36,right
2311,2018-12-30T21:25:32.300,45.45,25.97,0.00,0.00,0.00,NaN,NaN,None,NaN,football,NaN,NaN,football,2,2018123000,36,right
2312,2018-12-30T21:25:32.400,45.45,25.97,0.00,0.00,0.00,NaN,NaN,None,NaN,football,NaN,NaN,football,3,2018123000,36,right
2313,2018-12-30T21:25:32.500,45.45,25.97,0.00,0.00,0.00,NaN,NaN,None,NaN,football,NaN,NaN,football,4,2018123000,36,right
2314,2018-12-30T21:25:32.600,45.45,25.97,0.00,0.00,0.00,NaN,NaN,None,NaN,football,NaN,NaN,football,5,2018123000,36,right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2410,2018-12-30T21:25:42.200,75.70,16.48,0.85,1.91,0.04,NaN,NaN,None,NaN,football,NaN,NaN,football,101,2018123000,36,right
2411,2018-12-30T21:25:42.300,75.64,16.49,0.67,1.67,0.06,NaN,NaN,None,NaN,football,NaN,NaN,football,102,2018123000,36,right
2412,2018-12-30T21:25:42.400,75.59,16.50,0.49,1.68,0.06,NaN,NaN,None,NaN,football,NaN,NaN,football,103,2018123000,36,right
2413,2018-12-30T21:25:42.500,75.55,16.51,0.30,2.11,0.04,NaN,NaN,None,NaN,football,NaN,NaN,football,104,2018123000,36,right


In [5]:
play_data = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/plays.csv")
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull()", engine='python', inplace=False)


In [6]:
def get_distances(x, player_pos):
    dist = round(np.linalg.norm(player_pos.values - x.values),3)
    return dist

In [23]:

pd.options.mode.chained_assignment = None

start_time = time.time()


# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
RECEIVED = False
FUMBLED = False
TACKLED = False
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    # if no returner id, don't include play
    if pd.isnull(returnerId):
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        

    #####################
    
    
    # shift x-axis based on target's endzone
    playDirection = play_df.loc[play_df.index[0],'playDirection']
    
    if playDirection == 'left':
        play_df['adj_x'] = play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 270) % 360
        play_df['adj_dir'] = (play_df['dir'] + 270) % 360
    if playDirection == 'right':
        play_df['adj_x'] = 120 - play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 90) % 360
        play_df['adj_dir'] = (play_df['dir'] + 90) % 360
        
        
    # set football's NaN values to zero
    play_df.loc[play_df.team == 'football', 'o'] = 0
    play_df.loc[play_df.team == 'football', 'dir'] = 0
    play_df.loc[play_df.team == 'football', 'adj_o'] = 0
    play_df.loc[play_df.team == 'football', 'adj_dir'] = 0
    
    play_df['adj_o'].round(2)
    play_df['adj_dir'].round(2)
    
    play_df['sin_adj_o'] = np.sin(play_df['adj_o'])
    play_df['cos_adj_o'] = np.cos(play_df['adj_o'])
    play_df['sin_adj_dir'] = np.sin(play_df['adj_dir'])
    play_df['cos_adj_dir'] = np.cos(play_df['adj_dir'])
    
    # set football's NaN values to zero
    play_df.loc[play_df.team == 'football', 'cos_adj_o'] = 0
    play_df.loc[play_df.team == 'football', 'cos_adj_dir'] = 0
    
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_dis'] = play_df.groupby('displayName').dis.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'next_adj_x'] = play_df.groupby('displayName').adj_x.shift(-1)
    play_df.loc[:, 'next_adj_y'] = play_df.groupby('displayName').adj_y.shift(-1)
    play_df.loc[:, 'next_adj_o'] = play_df.groupby('displayName').adj_o.shift(-1)
    play_df.loc[:, 'next_adj_dir'] = play_df.groupby('displayName').adj_dir.shift(-1)
    play_df.loc[:, 'next_sin_adj_o'] = play_df.groupby('displayName').sin_adj_o.shift(-1)
    play_df.loc[:, 'next_cos_adj_o'] = play_df.groupby('displayName').cos_adj_o.shift(-1)
    play_df.loc[:, 'next_sin_adj_dir'] = play_df.groupby('displayName').sin_adj_dir.shift(-1)
    play_df.loc[:, 'next_cos_adj_dir'] = play_df.groupby('displayName').cos_adj_dir.shift(-1)
    
    # remove NAs created by shifting; i.e. remove plays where there is no next play
    play_df.dropna(axis=0, how='any', subset=['x'], inplace=True)
    
    # create reward - defined as forward progress
    play_df['reward'] = round(play_df['next_x'] - play_df['x'], 3)
    
    #display(play_df)
    
    
    #cond = play_df.loc[:,['event', 'displayName', 'position', 'team', 'playDirection', 'next_adj_x', 'next_adj_y', 'next_s', 'next_a', 'next_o', 'next_dir']]
    
    #f = cond.query("displayName == 'football'")
    
    #display(f.iloc[0:50,])
    #events = list(np.unique(f.event))
    #print(f"Events for play {playId} at index {index}")
    #print(events)
    
    
    # get starting time of play
    starting_time = min(play_df['time'])
    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            other_team_name = 'home'
            if ball_carrier_team == 'home':
                other_team_name = 'away'
            
            # get distance from ball carrier
            ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
            time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['x','y']],))
        
            time_play_df['ball_carrier_bool'] = time_play_df.nflId != returnerId
            time_play_df['team_index'] = time_play_df.team == ball_carrier_team
            time_play_df.sort_values(by=["ball_carrier_bool","team_index","dist_from_ball_carrier"], inplace=True, ignore_index=True)
            
            #### 
            #Check to make sure ball carrier has football
            #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
            ####
            football_index = time_play_df.index[time_play_df.team == 'football']
            try:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
            except:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']
            
            
            event = list(np.unique(time_play_df.event))[0]
            if event == 'kick_received' or event == 'punt_received':
                RECEIVED = True
            if event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled':
                TACKLED = True

            if TACKLED or (not RECEIVED):
                continue
            
            # store info

            states = time_play_df.loc[:,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier']].to_numpy()
            football_pos = time_play_df.loc[time_play_df.team == 'football',['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier']].to_numpy()
            
            rowDict['season'] = season
            rowDict['gameId'] = gameId
            rowDict['playId'] = playId
            
            rowDict["playIndex"] = index
            rowDict["timeIndex"] = time_index
            rowDict["time"] = time_string
            rowDict["football_pos"] = football_pos
            # add current state
            rowDict['state'] = states
            # add future states
            next_state = time_play_df.loc[:,['next_x','next_y', 'next_dis', 'next_s', 'next_a', 'next_o', 'next_dir', 'next_adj_x', 'next_adj_y', 'next_adj_o', 'next_adj_dir', 'next_sin_adj_o', 'next_cos_adj_o', 'next_sin_adj_dir', 'next_cos_adj_dir']].to_numpy()
            rowDict['next_state'] = next_state
            # add reward
            try:
                rowDict['reward'] = time_play_df.loc[0,'reward']
            except:
                print("error getting reward")
                continue

            action = states[:,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier']] - next_state
            rowDict['action'] = action.round(2)

            dict_list.append(rowDict)
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)
    
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")

full_train_df = pd.DataFrame(dict_list)

full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

full_train_df = full_train_df.dropna(axis=0, how='any', inplace=False)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("with_bc_dist.pkl")

print(F"total time: {total_time}  seconds")

Play 0 out of 4494
Play 250 out of 4494
Could not get returner id for index 362 at game 2018100701 for play 4281: <class 'Exception'>
Play 500 out of 4494
Could not get returner id for index 502 at game 2018101410 for play 2172: <class 'Exception'>
Play 750 out of 4494
Play 1000 out of 4494
Could not get returner id for index 1096 at game 2018120212 for play 1876: <class 'Exception'>
Play 1250 out of 4494
Could not get returner id for index 1470 at game 2018123011 for play 36: <class 'Exception'>
Play 1500 out of 4494
Play 1750 out of 4494
Play 2000 out of 4494
Play 2250 out of 4494
Could not get returner id for index 2443 at game 2019112408 for play 3869: <class 'Exception'>
Play 2500 out of 4494
Could not get returner id for index 2736 at game 2019121512 for play 4390: <class 'Exception'>
Play 2750 out of 4494
Play 3000 out of 4494
Play 3250 out of 4494
Play 3500 out of 4494
random error: index 0 is out of bounds for axis 0 with size 0
random error: index 0 is out of bounds for axis 

In [60]:
'''
time_play_df = DF of a specific play at a specific time. 23 rows

playerId = id of player we want to train action for
'''
def store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, playerId):

    other_team_name = 'home'
    if ball_carrier_team == 'home':
        other_team_name = 'away'

    # get distance from ball carrier
    ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['x','y']],))

    time_play_df['ball_carrier_bool'] = time_play_df.nflId != returnerId
    time_play_df['player_bool'] = time_play_df.nflId != playerId
    time_play_df['football_bool'] = time_play_df.displayName != 'football'
    time_play_df['team_index'] = time_play_df.team == ball_carrier_team
    time_play_df.sort_values(by=["player_bool", "football_bool", "team_index", "dist_from_ball_carrier"], inplace=True, ignore_index=True)

    #### 
    #Check to make sure ball carrier has football
    #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
    ####
    football_index = time_play_df.index[time_play_df.team == 'football']
    try:
        football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
    except:
        football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']

    no_football_df = time_play_df.query("displayName != 'football'")

    time_play_df['min_teammate_dist'] = time_play_df.apply(min_teammate_distances, axis=1, args=(no_football_df,))
    time_play_df['min_opponent_dist'] = time_play_df.apply(min_opponent_distances, axis=1, args=(no_football_df,))

    # store info
    display(time_play_df)
    #print('columns')
    #print(time_play_df.displayName)

    states = time_play_df.loc[:,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']].to_numpy()
    football_pos = time_play_df.loc[time_play_df.team == 'football',['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']].to_numpy()

    rowDict['season'] = season
    rowDict['gameId'] = gameId
    rowDict['playId'] = playId

    rowDict["playIndex"] = index
    rowDict["timeIndex"] = time_index
    rowDict["time"] = time_string
    rowDict["football_pos"] = football_pos
    # add current state
    rowDict['state'] = states
    # add future states
    next_state = time_play_df.loc[:,['next_x','next_y', 'next_dis', 'next_s', 'next_a', 'next_o', 'next_dir', 'next_adj_x', 'next_adj_y', 'next_adj_o', 'next_adj_dir', 'next_sin_adj_o', 'next_cos_adj_o', 'next_sin_adj_dir', 'next_cos_adj_dir']].to_numpy()
    rowDict['next_state'] = next_state
    # add reward
    rowDict['reward'] = time_play_df.loc[0,'reward']

    action = states[:,:-3]- next_state
    rowDict['action'] = action.round(2)
    
    return rowDict

In [68]:
'''
teammate list
'''

pd.options.mode.chained_assignment = None

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
RECEIVED = False
FUMBLED = False
TACKLED = False
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    # if no returner id, don't include play
    if pd.isnull(returnerId):
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        

    #####################
    
    # shift x-axis based on target's endzone
    playDirection = play_df.loc[play_df.index[0],'playDirection']
    
    if playDirection == 'left':
        play_df['adj_x'] = play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 270) % 360
        play_df['adj_dir'] = (play_df['dir'] + 270) % 360
    if playDirection == 'right':
        play_df['adj_x'] = 120 - play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 90) % 360
        play_df['adj_dir'] = (play_df['dir'] + 90) % 360
        
        
    # set football's NaN values to zero
    play_df.loc[play_df.team == 'football', 'o'] = 0
    play_df.loc[play_df.team == 'football', 'dir'] = 0
    play_df.loc[play_df.team == 'football', 'adj_o'] = 0
    play_df.loc[play_df.team == 'football', 'adj_dir'] = 0
    
    play_df['adj_o'].round(2)
    play_df['adj_dir'].round(2)
    
    play_df['sin_adj_o'] = np.sin(play_df['adj_o'])
    play_df['cos_adj_o'] = np.cos(play_df['adj_o'])
    play_df['sin_adj_dir'] = np.sin(play_df['adj_dir'])
    play_df['cos_adj_dir'] = np.cos(play_df['adj_dir'])
    
    # set football's NaN values to zero
    play_df.loc[play_df.team == 'football', 'cos_adj_o'] = 0
    play_df.loc[play_df.team == 'football', 'cos_adj_dir'] = 0
    
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_dis'] = play_df.groupby('displayName').dis.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'next_adj_x'] = play_df.groupby('displayName').adj_x.shift(-1)
    play_df.loc[:, 'next_adj_y'] = play_df.groupby('displayName').adj_y.shift(-1)
    play_df.loc[:, 'next_adj_o'] = play_df.groupby('displayName').adj_o.shift(-1)
    play_df.loc[:, 'next_adj_dir'] = play_df.groupby('displayName').adj_dir.shift(-1)
    play_df.loc[:, 'next_sin_adj_o'] = play_df.groupby('displayName').sin_adj_o.shift(-1)
    play_df.loc[:, 'next_cos_adj_o'] = play_df.groupby('displayName').cos_adj_o.shift(-1)
    play_df.loc[:, 'next_sin_adj_dir'] = play_df.groupby('displayName').sin_adj_dir.shift(-1)
    play_df.loc[:, 'next_cos_adj_dir'] = play_df.groupby('displayName').cos_adj_dir.shift(-1)
    
    # remove NAs created by shifting; i.e. remove plays where there is no next play
    play_df.dropna(axis=0, how='any', subset=['x'], inplace=True)
    
    # create reward - defined as forward progress
    play_df['reward'] = round(play_df['next_x'] - play_df['x'], 3)
    
    
    #cond = play_df.loc[:,['event', 'displayName', 'position', 'team', 'playDirection', 'next_adj_x', 'next_adj_y', 'next_s', 'next_a', 'next_o', 'next_dir']]
    
    #f = cond.query("displayName == 'football'")
    
    #display(f.iloc[0:50,])
    #events = list(np.unique(f.event))
    #print(f"Events for play {playId} at index {index}")
    #print(events)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            # determine whether we keep the event or not (i.e. if the ball carrier has the ball)
            event = list(np.unique(time_play_df.event))[0]
            if event == 'kick_received' or event == 'punt_received':
                RECEIVED = True
            if event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled':
                TACKLED = True
            if TACKLED or (not RECEIVED):
                continue
                
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            # get list of teammates
            teammate_id_list = time_play_df.query("team == @ball_carrier_team and displayName != 'football' and nflId != @returnerId").nflId
            # for each teammate, store as one training entry
            for playerId in teammate_id_list:

                rowDict = store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, playerId)
            
                dict_list.append(rowDict)
                
                break
            
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)
    
        break
    
    if index == 0:
        break

full_train_df = pd.DataFrame(dict_list)

#full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

#full_train_df = full_train_df.dropna(axis=0, how='any', inplace=False)



10


,time,x,y,s,a,dis,o,dir,event,nflId,...,next_sin_adj_dir,next_cos_adj_dir,reward,dist_from_ball_carrier,ball_carrier_bool,player_bool,football_bool,team_index,min_teammate_dist,min_opponent_dist
0,2018-09-07T01:34:34.500,39.07,41.27,2.59,1.39,0.25,99.09,341.49,kick_received,40408.0,...,-0.497743,-0.867324,-0.08,27.565,True,False,True,True,3.308,4.725
1,2018-09-07T01:34:34.500,11.15,42.28,2.91,6.78,0.17,0.00,0.00,kick_received,NaN,...,0.000000,0.000000,0.36,0.416,True,True,False,False,30.739,0.416
2,2018-09-07T01:34:34.500,40.75,33.99,9.31,1.89,0.94,281.50,280.27,kick_received,44962.0,...,0.273617,-0.961839,-0.91,30.420,True,True,True,False,1.777,5.861
3,2018-09-07T01:34:34.500,41.11,32.25,9.01,1.72,0.91,292.28,288.58,kick_received,46263.0,...,-0.695893,-0.718145,-0.85,31.289,True,True,True,False,1.777,4.805
4,2018-09-07T01:34:34.500,43.08,38.77,8.97,2.14,0.91,279.98,287.29,kick_received,46269.0,...,-0.899953,-0.435986,-0.85,31.764,True,True,True,False,3.450,4.725
5,2018-09-07T01:34:34.500,44.18,42.04,8.92,2.05,0.90,286.72,278.30,kick_received,44987.0,...,0.271009,0.962577,-0.86,32.653,True,True,True,False,3.450,4.992
6,2018-09-07T01:34:34.500,45.07,49.79,9.18,0.44,0.91,251.62,270.07,kick_received,42450.0,...,-0.881589,-0.472017,-0.92,34.334,True,True,True,False,6.487,7.453
7,2018-09-07T01:34:34.500,43.95,27.63,8.96,1.64,0.90,286.85,293.31,kick_received,44887.0,...,0.003522,-0.999994,-0.82,35.647,True,True,True,False,2.715,5.752
8,2018-09-07T01:34:34.500,44.11,24.92,10.05,0.76,1.01,299.74,292.21,kick_received,38707.0,...,0.642740,0.766084,-0.94,36.997,True,True,True,False,2.715,5.042
9,2018-09-07T01:34:34.500,43.90,21.75,9.11,1.06,0.91,283.77,289.89,kick_received,40191.0,...,-0.783117,-0.621874,-0.87,38.423,True,True,True,False,3.177,5.584


In [70]:
display(full_train_df)

,season,gameId,playId,playIndex,timeIndex,time,football_pos,state,next_state,reward,action
0,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[39.07, 41.27, 0.25, 2.59, 1.39, 99.09, 341.4...","[[38.99, 41.53, 0.26, 2.64, 1.24, 100.35, 344....",-0.08,"[[0.08, -0.26, -0.01, -0.05, 0.15, -1.26, -3.5..."


In [69]:
def min_teammate_distances(player_of_interest_row, time_play_df):
    team = player_of_interest_row['team_index']
    nflid = player_of_interest_row['nflId']
    new_df = time_play_df.query("team_index == @team and displayName != 'football' and nflId != @nflid")
    dists = new_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(player_of_interest_row[['x','y']],))
    min_teammate_dist = min(dists)
    return min_teammate_dist

In [49]:
def min_opponent_distances(player_of_interest_row, time_play_df):
    team = player_of_interest_row['team_index']
    nflid = player_of_interest_row['nflId']
    new_df = time_play_df.query("team_index != @team and displayName != 'football' and nflId != @nflid")
    dists = new_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(player_of_interest_row[['x','y']],))
    min_teammate_dist = min(dists)
    return min_teammate_dist

In [44]:
def get_distances(x, player_pos):
    dist = round(np.linalg.norm(player_pos.values - x.values),3)
    return dist

In [52]:
player = time_play_df.iloc[0,:]
#display(player)

#min_teammate_distances(time_play_df.iloc[0,:], time_play_df)

# do not want football in player calculations
no_football_df = time_play_df.query("displayName != 'football'")

time_play_df['min_teammate_dist'] = time_play_df.apply(min_teammate_distances, axis=1, args=(no_football_df,))
time_play_df['min_opponent_dist'] = time_play_df.apply(min_opponent_distances, axis=1, args=(no_football_df,))

In [53]:
display(time_play_df)

,time,x,y,s,a,dis,o,dir,event,nflId,...,next_sin_adj_o,next_cos_adj_o,next_sin_adj_dir,next_cos_adj_dir,reward,dist_from_ball_carrier,ball_carrier_bool,team_index,min_teammate_dist,min_opponent_dist
0,2018-09-07T01:34:34.500,11.53,42.45,1.32,3.03,0.14,117.30,148.06,kick_received,44979.0,...,-0.653910,0.756572,-0.990511,-0.137435,0.09,0.000,False,True,10.231,30.420
1,2018-09-07T01:34:34.500,11.15,42.28,2.91,6.78,0.17,0.00,0.00,kick_received,NaN,...,0.000000,0.000000,0.000000,0.000000,0.36,0.416,True,False,30.739,0.416
2,2018-09-07T01:34:34.500,40.75,33.99,9.31,1.89,0.94,281.50,280.27,kick_received,44962.0,...,0.930010,0.367534,0.273617,-0.961839,-0.91,30.420,True,False,1.777,5.861
3,2018-09-07T01:34:34.500,41.11,32.25,9.01,1.72,0.91,292.28,288.58,kick_received,46263.0,...,-0.825975,0.563707,-0.695893,-0.718145,-0.85,31.289,True,False,1.777,4.805
4,2018-09-07T01:34:34.500,43.08,38.77,8.97,2.14,0.91,279.98,287.29,kick_received,46269.0,...,-0.994179,-0.107739,-0.899953,-0.435986,-0.85,31.764,True,False,3.450,4.725
5,2018-09-07T01:34:34.500,44.18,42.04,8.92,2.05,0.90,286.72,278.30,kick_received,44987.0,...,-0.996054,0.088750,0.271009,0.962577,-0.86,32.653,True,False,3.450,4.992
6,2018-09-07T01:34:34.500,45.07,49.79,9.18,0.44,0.91,251.62,270.07,kick_received,42450.0,...,-0.985241,-0.171175,-0.881589,-0.472017,-0.92,34.334,True,False,6.487,7.453
7,2018-09-07T01:34:34.500,43.95,27.63,8.96,1.64,0.90,286.85,293.31,kick_received,44887.0,...,-0.866798,0.498660,0.003522,-0.999994,-0.82,35.647,True,False,2.715,5.752
8,2018-09-07T01:34:34.500,44.11,24.92,10.05,0.76,1.01,299.74,292.21,kick_received,38707.0,...,-0.748048,0.663645,0.642740,0.766084,-0.94,36.997,True,False,2.715,5.042
9,2018-09-07T01:34:34.500,43.90,21.75,9.11,1.06,0.91,283.77,289.89,kick_received,40191.0,...,0.796362,0.604821,-0.783117,-0.621874,-0.87,38.423,True,False,3.177,5.584
